# IVS R-diff quicklook

Minimal notebook that mirrors the MATLAB `Plot_script_sample.m` workflow: load two IVS skill files, compute D2–D1 R differences, plot on the M36 EASE grid, and save a `.mat` file with `Rdiff_vector`, `lats`, and `lons`.


In [1]:
from pathlib import Path
import sys

import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat, savemat

# Hardwired repo path so the EASE utility is available
REPO_ROOT = Path("/Users/amfox/Desktop/geosldas-analysis/projects/matlab2python")
sys.path.append(str(REPO_ROOT / "shared" / "python"))
from EASEv2 import EASEv2_ind2latlon  # noqa: E402


In [ ]:
# Example inputs and outputs (edit paths/versions if needed)
data_dir = Path("/Users/amfox/for_Andy/IVs")
output_dir = Path("/Users/amfox/Desktop/geosldas-analysis/projects/matlab2python/scripts")

D1_version = "OLv7_M36_MULTI_type_13_comb_fp_scaled"
D2_version = "DAv7_M36_ASCAT_type_13_comb_fp_scaled"

f_D1 = data_dir / f"SMPL3_{D1_version}_IVD_IVS_stats_lag2day_201504_202103.mat"
f_D2 = data_dir / f"SMPL3_{D2_version}_IVD_IVS_stats_lag2day_201504_202103.mat"

f_D1, f_D2


In [ ]:
def _as_vector(mat_array: np.ndarray) -> np.ndarray:
    return np.asarray(mat_array, dtype=float).ravel(order="F")


def load_skill_file(path: Path) -> tuple[np.ndarray, np.ndarray]:
    data = loadmat(path, simplify_cells=True)
    return _as_vector(data["R2_ivs_obs"]), _as_vector(data["R2_ivs_mod"])


R2_obs_D1, R2_mod_D1 = load_skill_file(f_D1)
R2_obs_D2, R2_mod_D2 = load_skill_file(f_D2)

R_D1 = np.sqrt(R2_mod_D1)
R_D2 = np.sqrt(R2_mod_D2)
R_OBS = np.sqrt(R2_obs_D2)

R_D1[R_D1 < 0.1] = np.nan
R_D2[R_D2 < 0.1] = np.nan
R_OBS[R_OBS < 0.1] = np.nan
R_D1[np.isnan(R_OBS)] = np.nan
R_D2[np.isnan(R_OBS)] = np.nan

Rdiff_vector = R_D2 - R_D1
Rdiff_vector


In [ ]:
# Build EASEv2 M36 lat/lon grids
rows = np.arange(406)
cols = np.arange(964)
lat, lon = EASEv2_ind2latlon(rows, cols, "M36")
lon_grid = np.tile(lon[:, None], (1, lat.size))
lat_grid = np.tile(lat, (lon.size, 1))

Rdiff_grid = np.reshape(Rdiff_vector, lon_grid.shape, order="F")
Rdiff_grid


In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(12, 5))
mesh = ax.pcolormesh(lon_grid, lat_grid, Rdiff_grid, shading="flat")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_title(f"R diff: {D2_version} minus {D1_version}")
fig.colorbar(mesh, ax=ax, label="R difference (D2 - D1)")
plt.tight_layout()
plt.show()


In [ ]:
# Save .mat output
outname = f"Rdiff_{D2_version}_minus_{D1_version}.mat"
output_dir.mkdir(parents=True, exist_ok=True)
savemat(
    output_dir / outname,
    {
        "Rdiff_vector": Rdiff_vector,
        "lats": lat_grid.reshape(-1, order="F"),
        "lons": lon_grid.reshape(-1, order="F"),
    },
)
output_dir / outname
